In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
#Libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from matplotlib.pyplot import figure
import pyodbc 
import glob

In [2]:
# Must include MGRA
columns_to_check = ['mgra', 'pop', 'hhp', 'gq_civ', 'gq_mil']

In [3]:
config_filename = 'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/ds41_42_config.yml'

In [4]:
with open(config_filename, "r") as yml_file:
    config = yaml.safe_load(yml_file)

In [5]:
DS38_datafiles = ['DS38_2016', 'DS38_2018', 'DS38_2020', 'DS38_2023', 'DS38_2025', 'DS38_2026', 'DS38_2029', 'DS38_2030', 'DS38_2032', 'DS38_2035', 'DS38_2040', 'DS38_2045', 'DS38_2050']

In [6]:
DS42_datafiles = ['DS42_2016', 'DS42_2018', 'DS42_2020', 'DS42_2023', 'DS42_2025', 'DS42_2026', 'DS42_2029', 'DS42_2030', 'DS42_2032', 'DS42_2035', 'DS42_2040', 'DS42_2045', 'DS42_2050']

In [7]:
older_DS = 'DS38'
current_DS = 'DS42'

# Creating the MGRA level Data Frame

In [8]:
# Older Data Frame
DS38_Data = pd.DataFrame(columns = columns_to_check)
for file_name in DS38_datafiles:
    working_df = pd.read_csv(config[older_DS][file_name], usecols=columns_to_check)
    working_df['year'] = f"{file_name[-4:]}"
    DS38_Data = DS38_Data.append(working_df)

In [9]:
#Making it original 
DS38_Data.columns = [x + '_DS38' for x in DS38_Data.columns]

In [10]:
# Newer Data Frame
DS42_Data = pd.DataFrame(columns = columns_to_check)
for file_name in DS42_datafiles:
    working_df = pd.read_csv(config[current_DS][file_name], usecols=columns_to_check)
    working_df['year'] = f"{file_name[-4:]}"
    DS42_Data = DS42_Data.append(working_df)

In [11]:
# making it original
DS42_Data.columns = [x + '_DS42' for x in DS42_Data.columns]

In [12]:
# Combine to make one Data Frame
mgra_df = pd.concat([DS38_Data, DS42_Data], axis=1)

In [13]:
# Clean up MGRA DF
mgra_df = mgra_df.rename(columns={'mgra_DS38':'mgra'})
mgra_df = mgra_df.drop(['mgra_DS42', 'year_DS38'], axis=1)
mgra_df = mgra_df.groupby(['mgra', 'year_DS42']).sum()

In [14]:
mgra_df

pop_DS38  hhp_DS38  gq_civ_DS38  gq_mil_DS38  pop_DS42  \
mgra  year_DS42                                                           
1     2016             41        41            0            0        41   
      2018             41        41            0            0        44   
      2020             47        47            0            0        44   
      2023             38        38            0            0        46   
      2025             35        35            0            0        45   
...                   ...       ...          ...          ...       ...   
23002 2032              0         0            0            0         0   
      2035              0         0            0            0         0   
      2040              0         0            0            0         0   
      2045              0         0            0            0         0   
      2050              0         0            0            0         0   

                 hhp_DS42  gq_civ_DS42  gq_mil_DS42  
mgra  year_DS42                                      
1     2016             41            0            0  
      2018             44            0            0  
      2020             44            0            0  
      2023             46            0            0  
      2025             45            0            0  
...                   ...          ...          ...  
23002 2032              0            0            0  
      2035              0            0            0  
      2040              0            0            0  
      2045              0            0            0  
      2050              0            0            0  

[299026 rows x 8 columns]

In [15]:
hhp_diff = (mgra_df['hhp_DS42'] - mgra_df['hhp_DS38']).to_frame(name='hhp_diff')
large_diff_hhp = hhp_diff[(hhp_diff['hhp_diff'] > 500) | (hhp_diff['hhp_diff'] < -500)]

# Grabbing The Data From SQL

In [16]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DDAMWSQL16.sandag.org;'
                      'Database=demographic_warehouse;'
                      'Trusted_Connection=yes;')

In [17]:
query_all = "SELECT mgra, geotype, geozone FROM demographic_warehouse.dim.mgra WHERE series = 14 AND (geotype='cpa' OR geotype='jurisdiction')" #Remove the last and part when I do this for real 

In [18]:
sql_query = pd.read_sql_query(query_all,conn)

sql_df_all = pd.DataFrame(sql_query)

In [19]:
sql_df_all

,mgra,geotype,geozone
0,614,cpa,College Area
1,614,jurisdiction,San Diego
2,614,cpa,Mid-City:Kensington-Talmadge
3,614,jurisdiction,San Diego
4,615,cpa,Mid-City:Kensington-Talmadge
...,...,...,...
54627,7899,jurisdiction,Chula Vista
54628,7900,cpa,*Not in a CPA*
54629,7900,jurisdiction,Chula Vista
54630,7901,cpa,*Not in a CPA*


# Creating the CPA DF

In [20]:
SQL_CPA = sql_df_all[sql_df_all['geotype']=='cpa']
SQL_CPA = SQL_CPA.rename(columns={'geozone':'cpa_geozone'})
SQL_CPA = SQL_CPA.drop(['geotype'], axis = 1)

In [21]:
#DS38
DS38_Data_cpa = DS38_Data.merge(SQL_CPA, how='left', left_on='mgra_DS38', right_on='mgra')
DS38_Data_cpa = DS38_Data_cpa.drop(['mgra'], axis=1)
DS38_Data_cpa = DS38_Data_cpa.rename(columns={'cpa_geozone':'cpa_geozone_DS38'})

In [22]:
# DS42
DS42_Data_cpa = DS42_Data.merge(SQL_CPA, how='left', left_on='mgra_DS42', right_on='mgra')
DS42_Data_cpa = DS42_Data_cpa.drop(['mgra'], axis=1)
#DS42_Data_cpa.columns = [x + '_DS42' for x in DS42_Data_cpa.columns]

In [23]:
# Combine to make one Data Frame
cpa_df = pd.concat([DS38_Data_cpa, DS42_Data_cpa], axis=1, join='inner')


In [24]:
# Clean up CPA DF
cpa_df = cpa_df.drop(['mgra_DS38', 'cpa_geozone_DS38', 'year_DS38', 'mgra_DS42'], axis=1)
cpa_df = cpa_df.groupby(['cpa_geozone', 'year_DS42']).sum()
cpa_df = cpa_df.drop('*Not in a CPA*') #Per meeting with Purva and Mike I was instructed that I can just drop the MGRAs that do not lie in a cpa

In [25]:
# Final CPA DF
cpa_df.head(50)

pop_DS38  hhp_DS38  gq_civ_DS38  \
cpa_geozone               year_DS42                                    
32nd Street Naval Station 2016           8159       573            0   
                          2018           8503       574            0   
                          2020           9981       542            0   
                          2023          12549       845            0   
                          2025          13460      1001            0   
                          2026          13499      1040            0   
                          2029          13601      1142            0   
                          2030          13609      1150            0   
                          2032          13719      1260            0   
                          2035          13890      1431            0   
                          2040          13958      1499            0   
                          2045          14017      1558            0   
                          2050          13950      1491            0   
Alpine                    2016          20788     20533          255   
                          2018          20862     20577          285   
                          2020          20754     20469          285   
                          2023          20457     20172          285   
                          2025          20291     20006          285   
                          2026          20199     19914          285   
                          2029          19969     19684          285   
                          2030          19870     19585          285   
                          2032          19688     19403          285   
                          2035          19545     19260          285   
                          2040          19600     19315          285   
                          2045          19832     19547          285   
                          2050          19902     19617          285   
Balboa Park               2016           6296      5958          182   
                          2018           6746      6252          192   
                          2020           6855      6361          192   
                          2023           7166      6672          192   
                          2025           7446      6952          192   
                          2026           7466      6972          192   
                          2029           7690      7196          192   
                          2030           7772      7278          192   
                          2032           7761      7267          192   
                          2035           7743      7249          192   
                          2040           8162      7668          192   
                          2045           8291      7797          192   
                          2050           8367      7873          192   
Barona                    2016           2667      2666            1   
                          2018           2678      2677            1   
                          2020           2700      2699            1   
                          2023           2622      2621            1   
                          2025           2631      2630            1   
                          2026           2579      2578            1   
                          2029           2539      2538            1   
                          2030           2538      2537            1   
                          2032           2557      2556            1   
                          2035           2546      2545            1   
                          2040           2587      2586            1   

                                     gq_mil_DS38  pop_DS42  hhp_DS42  \
cpa_geozone               year_DS42                                    
32nd Street Naval Station 2016              7586      8159       573   
                          2018              7929      8509       580   
    

# Creating the Jurisdiction DF

In [26]:
SQL_jur = sql_df_all[sql_df_all['geotype']=='jurisdiction']
SQL_jur = SQL_jur.rename(columns={'geozone':'jur_geozone'})
SQL_jur = SQL_jur.drop(['geotype'], axis = 1)

In [27]:
#DS38
DS38_Data_jur = DS38_Data.merge(SQL_jur, how='left', left_on='mgra_DS38', right_on='mgra')
DS38_Data_jur = DS38_Data_jur.drop(['mgra'], axis=1)
DS38_Data_jur = DS38_Data_jur.rename(columns={'jur_geozone':'jur_geozone_DS38'})

In [28]:
#DS42
DS42_Data_jur = DS42_Data.merge(SQL_jur, how='left', left_on='mgra_DS42', right_on='mgra')
DS42_Data_jur = DS42_Data_jur.drop(['mgra'], axis=1)

In [29]:
jur_df = pd.concat([DS38_Data_jur, DS42_Data_jur], axis=1, join='inner')

In [30]:
# Ensure JurZones Are Equal, this should be zero as it is counting the number of falses  
sum(jur_df['jur_geozone_DS38'] == jur_df['jur_geozone']) - len(jur_df)

0

In [31]:
# Clean up JUR DF
jur_df = jur_df.drop(['mgra_DS38', 'jur_geozone_DS38', 'year_DS38', 'mgra_DS42'], axis=1)
jur_df = jur_df.groupby(['jur_geozone', 'year_DS42']).sum()

In [32]:
# Final Jurisdiction DF
jur_df.head(30)

pop_DS38  hhp_DS38  gq_civ_DS38  gq_mil_DS38  pop_DS42  \
jur_geozone year_DS42                                                           
Carlsbad    2016         125491    124576          915            0    125491   
            2018         127537    126622          915            0    127505   
            2020         128615    127700          915            0    128573   
            2023         128733    127818          915            0    128716   
            2025         128640    127725          915            0    128549   
            2026         128751    127836          915            0    128722   
            2029         129068    128153          915            0    129211   
            2030         129225    128310          915            0    129359   
            2032         129514    128599          915            0    129602   
            2035         131803    130888          915            0    131853   
            2040         132985    132070          915            0    133088   
            2045         133804    132889          915            0    133883   
            2050         134407    133492          915            0    134329   
Chula Vista 2016         272502    270799         1703            0    272502   
            2018         274751    273031         1720            0    274709   
            2020         274820    273100         1720            0    274808   
            2023         284890    283170         1720            0    284853   
            2025         292060    290340         1720            0    292024   
            2026         291984    290264         1720            0    291957   
            2029         291906    290186         1720            0    291991   
            2030         291972    290252         1720            0    292010   
            2032         293039    291319         1720            0    293114   
            2035         295182    293462         1720            0    295259   
            2040         307114    305394         1720            0    307080   
            2045         322452    320732         1720            0    322366   
            2050         330470    328750         1720            0    330508   
Coronado    2016          25369     18803          149         6417     25369   
            2018          25481     18915          149         6417     25477   
            2020          25522     18956          149         6417     25506   
            2023          25725     19159          149         6417     25724   

                       hhp_DS42  gq_civ_DS42  gq_mil_DS42  
jur_geozone year_DS42                                      
Carlsbad    2016         124576          915            0  
            2018         126590          915            0  
            2020         127658          915            0  
            2023         127801          915            0  
            2025         127634          915            0  
            2026         127807          915            0  
            2029         128296          915            0  
            2030         128444          915            0  
            2032         128687          915            0  
            2035         130938          915            0  
            2040         132173          915            0  
            2045         132968          915            0  
            2050         133414          915            0  
Chula Vista 2016         270799         1703            0  
            2018         272989         1720            0  
            2020         273088         1720            0  
            2023         283133         1720            0  
            2025         290304         1720            0  
            2026         290237         1720            0  
            2029         290271         1720            0  
            2030         290290         1720            0  
            2032         291394         1720      

# Grabbing the DOF Data 

In [33]:
#dof_data = pd.read_excel(config['dof']['dof_data'])

# MGRA Analysis

In [34]:
mgra_diff_df = pd.DataFrame(index=mgra_df.index)

In [35]:
#MGRA Diff
mgra_diff_df['pop_diff'] = mgra_df['pop_DS42'] - mgra_df['pop_DS38']
mgra_diff_df['hhp_diff'] = mgra_df['hhp_DS42'] - mgra_df['hhp_DS38']
mgra_diff_df['gq_civ_diff'] = mgra_df['gq_civ_DS42'] - mgra_df['gq_civ_DS38']
mgra_diff_df['gq_mil_diff'] = mgra_df['gq_mil_DS42'] - mgra_df['gq_mil_DS38']

In [93]:
mgra_diff_df['pop_diff']

mgra   year_DS42
1      2016          0
       2018          3
       2020         -3
       2023          8
       2025         10
                    ..
23002  2032          0
       2035          0
       2040          0
       2045          0
       2050          0
Name: pop_diff, Length: 299026, dtype: int64

In [94]:
mgra_df

pop_DS38  hhp_DS38  gq_civ_DS38  gq_mil_DS38  pop_DS42  \
mgra  year_DS42                                                           
1     2016             41        41            0            0        41   
      2018             41        41            0            0        44   
      2020             47        47            0            0        44   
      2023             38        38            0            0        46   
      2025             35        35            0            0        45   
...                   ...       ...          ...          ...       ...   
23002 2032              0         0            0            0         0   
      2035              0         0            0            0         0   
      2040              0         0            0            0         0   
      2045              0         0            0            0         0   
      2050              0         0            0            0         0   

                 hhp_DS42  gq_civ_DS42  gq_mil_DS42  
mgra  year_DS42                                      
1     2016             41            0            0  
      2018             44            0            0  
      2020             44            0            0  
      2023             46            0            0  
      2025             45            0            0  
...                   ...          ...          ...  
23002 2032              0            0            0  
      2035              0            0            0  
      2040              0            0            0  
      2045              0            0            0  
      2050              0            0            0  

[299026 rows x 8 columns]

In [91]:
# mgra_diff_df.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4238_v2_results/MGRA/DS4238_diff.csv', index = True)

In [37]:
# MGRA Stats 
mgra_stats = pd.DataFrame(columns=mgra_diff_df.columns, index=['Max', 'Min', 'Average'])
for col in mgra_stats.columns:
    mgra_stats[col]['Max'] = np.max(list(mgra_diff_df[col]))
    mgra_stats[col]['Min'] = np.min(list(mgra_diff_df[col]))
    mgra_stats[col]['Average'] = np.mean(list(mgra_diff_df[col]))

In [38]:
mgra_stats.head()


,pop_diff,hhp_diff,gq_civ_diff,gq_mil_diff
Max,214,214,0,0
Min,-644,-644,0,0
Average,0.0,0.0,0.0,0.0


In [95]:
# mgra_stats.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4238_v2_results/MGRA/mgra_stats.csv', index = True)

In [97]:
# Major HHP Diffs
hhp_diff = (mgra_df['hhp_DS42'] - mgra_df['hhp_DS38']).to_frame(name='hhp_diff')
large_diff_hhp = hhp_diff[(hhp_diff['hhp_diff'] > 500) | (hhp_diff['hhp_diff'] < -500)]

In [98]:
# large_diff_hhp.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4238_v2_results/MGRA/large_diff_hhp.csv', index = True)

In [99]:
# Major Pop Diff
pop_diff = (mgra_df['pop_DS42'] - mgra_df['pop_DS38']).to_frame(name='pop_diff')
large_diff_pop = pop_diff[(pop_diff['pop_diff'] > 500) | (pop_diff['pop_diff'] < -500)]

In [100]:
# large_diff_pop.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4238_v2_results/MGRA/large_diff_pop.csv', index = True)


In [101]:
mgra_df

pop_DS38  hhp_DS38  gq_civ_DS38  gq_mil_DS38  pop_DS42  \
mgra  year_DS42                                                           
1     2016             41        41            0            0        41   
      2018             41        41            0            0        44   
      2020             47        47            0            0        44   
      2023             38        38            0            0        46   
      2025             35        35            0            0        45   
...                   ...       ...          ...          ...       ...   
23002 2032              0         0            0            0         0   
      2035              0         0            0            0         0   
      2040              0         0            0            0         0   
      2045              0         0            0            0         0   
      2050              0         0            0            0         0   

                 hhp_DS42  gq_civ_DS42  gq_mil_DS42  
mgra  year_DS42                                      
1     2016             41            0            0  
      2018             44            0            0  
      2020             44            0            0  
      2023             46            0            0  
      2025             45            0            0  
...                   ...          ...          ...  
23002 2032              0            0            0  
      2035              0            0            0  
      2040              0            0            0  
      2045              0            0            0  
      2050              0            0            0  

[299026 rows x 8 columns]

In [104]:
# # Percent Diff Between Years
# mgra_names = []
# for name in mgra_df.index:
#     mgra_names.append(name[0])
# unique_mgra_names = list(np.unique(np.array(mgra_names)))
# years = ['2016','2018','2020','2023','2025','2026','2029','2030','2032','2035','2040','2045','2050']

In [105]:
# pct_diff_df_mgra = pd.DataFrame(columns=mgra_df.columns, index=mgra_df.index)
# # Year over year increase 
# for zone_name in unique_mgra_names:
#     for i in range(1,len(years)):
#         for col in mgra_df.columns:
#             current_val = mgra_df.loc[zone_name].loc[years[i]][col]
#             prior_val = mgra_df.loc[zone_name].loc[years[i-1]][col]
#             percent_diff = ((current_val - prior_val)/prior_val)*100
#             pct_diff_df_mgra.loc[zone_name].loc[years[i]][col] = percent_diff
# pct_diff_df_mgra

C:\Users\cra\AppData\Local\Temp/ipykernel_12344/2314412121.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  percent_diff = ((current_val - prior_val)/prior_val)*100


KeyboardInterrupt: 

# Region Analysis

In [44]:
mgra_df

pop_DS38  hhp_DS38  gq_civ_DS38  gq_mil_DS38  pop_DS42  \
mgra  year_DS42                                                           
1     2016             41        41            0            0        41   
      2018             41        41            0            0        44   
      2020             47        47            0            0        44   
      2023             38        38            0            0        46   
      2025             35        35            0            0        45   
...                   ...       ...          ...          ...       ...   
23002 2032              0         0            0            0         0   
      2035              0         0            0            0         0   
      2040              0         0            0            0         0   
      2045              0         0            0            0         0   
      2050              0         0            0            0         0   

                 hhp_DS42  gq_civ_DS42  gq_mil_DS42  
mgra  year_DS42                                      
1     2016             41            0            0  
      2018             44            0            0  
      2020             44            0            0  
      2023             46            0            0  
      2025             45            0            0  
...                   ...          ...          ...  
23002 2032              0            0            0  
      2035              0            0            0  
      2040              0            0            0  
      2045              0            0            0  
      2050              0            0            0  

[299026 rows x 8 columns]

In [45]:
# mgra_names = []
# for name in mgra_df.index:
#     mgra_names.append(name[0])
# unique_mgra_names = list(np.unique(np.array(mgra_names)))
# years = ['2016','2018','2020','2023','2025','2026','2029','2030','2032','2035','2040','2045','2050']

In [46]:
# mgra_sums_df = pd.DataFrame(columns=mgra_df.columns, index=years)
# for col in mgra_df:
#     for year in years:
#             total = 0
#             for mgra in unique_mgra_names:
#                 total += int(mgra_df.loc[mgra].loc[[year]][col])
#             mgra_sums_df[col][year] = total

In [47]:
# mgra_sums_df

In [48]:
# mgra_sums_df.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS42_42/DS4235_results/mgra_sums.csv', index = True)

# CPA Analysis

In [49]:
cpa_df.head()

pop_DS38  hhp_DS38  gq_civ_DS38  \
cpa_geozone               year_DS42                                    
32nd Street Naval Station 2016           8159       573            0   
                          2018           8503       574            0   
                          2020           9981       542            0   
                          2023          12549       845            0   
                          2025          13460      1001            0   

                                     gq_mil_DS38  pop_DS42  hhp_DS42  \
cpa_geozone               year_DS42                                    
32nd Street Naval Station 2016              7586      8159       573   
                          2018              7929      8509       580   
                          2020              9439     10004       565   
                          2023             11704     12521       817   
                          2025             12459     13359       900   

                                     gq_civ_DS42  gq_mil_DS42  
cpa_geozone               year_DS42                            
32nd Street Naval Station 2016                 0         7586  
                          2018                 0         7929  
                          2020                 0         9439  
                          2023                 0        11704  
                          2025                 0        12459

In [50]:
cpa_diff_df = pd.DataFrame(index=cpa_df.index)

In [51]:
#41 minus 35
cpa_diff_df['pop_diff'] = cpa_df['pop_DS42'] - cpa_df['pop_DS38']
cpa_diff_df['hhp_diff'] = cpa_df['hhp_DS42'] - cpa_df['hhp_DS38']
cpa_diff_df['gq_civ_diff'] = cpa_df['gq_civ_DS42'] - cpa_df['gq_civ_DS38']
cpa_diff_df['gq_mil_diff'] = cpa_df['gq_mil_DS42'] - cpa_df['gq_mil_DS38']

In [106]:
# cpa_diff_df.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4238_v2_results/CPA/cpa_diff_df.csv', index = True)

In [53]:
cpa_diff_df.head(10)

pop_diff  hhp_diff  gq_civ_diff  \
cpa_geozone               year_DS42                                    
32nd Street Naval Station 2016              0         0            0   
                          2018              6         6            0   
                          2020             23        23            0   
                          2023            -28       -28            0   
                          2025           -101      -101            0   
                          2026           -105      -105            0   
                          2029           -142      -142            0   
                          2030           -100      -100            0   
                          2032           -140      -140            0   
                          2035           -243      -243            0   

                                     gq_mil_diff  
cpa_geozone               year_DS42               
32nd Street Naval Station 2016                 0  
                          2018                 0  
                          2020                 0  
                          2023                 0  
                          2025                 0  
                          2026                 0  
                          2029                 0  
                          2030                 0  
                          2032                 0  
                          2035                 0

In [54]:
# 41 minus 35
cpa_stats = pd.DataFrame(columns=cpa_diff_df.columns, index=['Max', 'Min', 'Average'])
for col in mgra_stats.columns:
    cpa_stats[col]['Max'] = np.max(list(cpa_diff_df[col]))
    cpa_stats[col]['Min'] = np.min(list(cpa_diff_df[col]))
    cpa_stats[col]['Average'] = np.mean(list(cpa_diff_df[col]))

In [55]:
cpa_stats

,pop_diff,hhp_diff,gq_civ_diff,gq_mil_diff
Max,1466,1466,0,0
Min,-1189,-1189,0,0
Average,7.641026,7.641026,0.0,0.0


In [107]:
# cpa_stats.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4238_v2_results/CPA/cpa_stats.csv', index = True)

## CPA Sums

In [57]:
CPA_names = []
for name in cpa_df.index:
    CPA_names.append(name[0])
unique_cpa_names = list(np.unique(np.array(CPA_names)))
years = ['2016','2018','2020','2023','2025','2026','2029','2030','2032','2035','2040','2045','2050']

In [58]:
cpa_sums_df = pd.DataFrame(columns=cpa_df.columns, index=years)
for col in cpa_df:
    for year in years:
            total = 0
            for cpa in unique_cpa_names:
                total += int(cpa_df.loc[cpa].loc[[year]][col])
            cpa_sums_df[col][year] = total


In [59]:
cpa_sums_df

,pop_DS38,hhp_DS38,gq_civ_DS38,gq_mil_DS38,pop_DS42,hhp_DS42,gq_civ_DS42,gq_mil_DS42
2016,2341482,2210596,60051,70835,2341482,2210596,60051,70835
2018,2381849,2237108,66793,77948,2381310,2236569,66793,77948
2020,2414402,2261514,70410,82478,2414322,2261434,70410,82478
2023,2449909,2288102,72534,89273,2449251,2287444,72534,89273
2025,2479444,2315206,72700,91538,2478743,2314505,72700,91538
2026,2497362,2333041,72783,91538,2497630,2333309,72783,91538
2029,2544149,2379579,73032,91538,2545578,2381008,73032,91538
2030,2557107,2392454,73115,91538,2559115,2394462,73115,91538
2032,2581695,2416876,73281,91538,2583341,2418522,73281,91538
2035,2619573,2454505,73530,91538,2621661,2456593,73530,91538


In [108]:
# cpa_sums_df.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4238_v2_results/CPA/cpa_sums_df.csv', index = True)

In [61]:
cpa_df

pop_DS38  hhp_DS38  gq_civ_DS38  \
cpa_geozone               year_DS42                                    
32nd Street Naval Station 2016           8159       573            0   
                          2018           8503       574            0   
                          2020           9981       542            0   
                          2023          12549       845            0   
                          2025          13460      1001            0   
...                                       ...       ...          ...   
Via De La Valle           2032           1214      1214            0   
                          2035           1250      1250            0   
                          2040           1291      1291            0   
                          2045           1287      1287            0   
                          2050           1251      1251            0   

                                     gq_mil_DS38  pop_DS42  hhp_DS42  \
cpa_geozone               year_DS42                                    
32nd Street Naval Station 2016              7586      8159       573   
                          2018              7929      8509       580   
                          2020              9439     10004       565   
                          2023             11704     12521       817   
                          2025             12459     13359       900   
...                                          ...       ...       ...   
Via De La Valle           2032                 0      1131      1131   
                          2035                 0      1176      1176   
                          2040                 0      1192      1192   
                          2045                 0      1230      1230   
                          2050                 0      1231      1231   

                                     gq_civ_DS42  gq_mil_DS42  
cpa_geozone               year_DS42                            
32nd Street Naval Station 2016                 0         7586  
                          2018                 0         7929  
                          2020                 0         9439  
                          2023                 0        11704  
                          2025                 0        12459  
...                                          ...          ...  
Via De La Valle           2032                 0            0  
                          2035                 0            0  
                          2040                 0            0  
                          2045                 0            0  
                          2050                 0            0  

[1131 rows x 8 columns]

In [62]:
# Major HHP Diffs
cpa_hhp_diff = (cpa_df['hhp_DS42'] - cpa_df['hhp_DS38']).to_frame(name='hhp_diff')
cpa_large_diff_hhp = cpa_hhp_diff[(cpa_hhp_diff['hhp_diff'] > 750) | (cpa_hhp_diff['hhp_diff'] < -750)]

In [109]:
# cpa_large_diff_hhp.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4238_v2_results/CPA/cpa_large_diff_hhp.csv', index = True)

In [64]:
# Major pop Diffs
cpa_pop_diff = (cpa_df['pop_DS42'] - cpa_df['pop_DS38']).to_frame(name='pop_diff')
cpa_large_diff_pop = cpa_pop_diff[(cpa_pop_diff['pop_diff'] > 500) | (cpa_pop_diff['pop_diff'] < -500)]

In [110]:
# cpa_large_diff_pop.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4238_v2_results/CPA/cpa_large_diff_pop.csv', index = True)

In [112]:
# Percent Differences YoY
cpa_names = []
for name in cpa_df.index:
    cpa_names.append(name[0])
unique_cpa_names = list(np.unique(np.array(cpa_names)))
years = ['2016','2018','2020','2023','2025','2026','2029','2030','2032','2035','2040','2045','2050']


pct_diff_df_cpa = pd.DataFrame(columns=cpa_df.columns, index=cpa_df.index)
# Year over year increase 
for zone_name in unique_cpa_names:
    for i in range(1,len(years)):
        for col in cpa_df.columns:
            current_val = cpa_df.loc[zone_name].loc[years[i]][col]
            prior_val = cpa_df.loc[zone_name].loc[years[i-1]][col]
            percent_diff = ((current_val - prior_val)/prior_val)*100
            pct_diff_df_cpa.loc[zone_name].loc[years[i]][col] = percent_diff
pct_diff_df_cpa

Maximum_percent_difference = pd.DataFrame(pct_diff_df_cpa.max())
Minimum_percent_difference = pd.DataFrame(pct_diff_df_cpa.min())

C:\Users\cra\AppData\Local\Temp/ipykernel_12344/3726677170.py:16: RuntimeWarning: invalid value encountered in longlong_scalars
  percent_diff = ((current_val - prior_val)/prior_val)*100


In [127]:
pct_diff_df_cpa

pop_DS38   hhp_DS38 gq_civ_DS38  \
cpa_geozone               year_DS42                                     
32nd Street Naval Station 2016             NaN        NaN         NaN   
                          2018        4.216203    0.17452         NaN   
                          2020         17.3821  -5.574913         NaN   
                          2023       25.728885  55.904059         NaN   
                          2025        7.259543  18.461538         NaN   
...                                        ...        ...         ...   
Via De La Valle           2032        0.998336   0.998336         NaN   
                          2035        2.965404   2.965404         NaN   
                          2040            3.28       3.28         NaN   
                          2045       -0.309837  -0.309837         NaN   
                          2050       -2.797203  -2.797203         NaN   

                                    gq_mil_DS38   pop_DS42   hhp_DS42  \
cpa_geozone               year_DS42                                     
32nd Street Naval Station 2016              NaN        NaN        NaN   
                          2018         4.521487   4.289741    1.22164   
                          2020        19.044016  17.569632  -2.586207   
                          2023        23.996186  25.159936   44.60177   
                          2025         6.450786   6.692756  10.159119   
...                                         ...        ...        ...   
Via De La Valle           2032              NaN   1.162791   1.162791   
                          2035              NaN    3.97878    3.97878   
                          2040              NaN   1.360544   1.360544   
                          2045              NaN   3.187919   3.187919   
                          2050              NaN   0.081301   0.081301   

                                    gq_civ_DS42 gq_mil_DS42  
cpa_geozone               year_DS42                          
32nd Street Naval Station 2016              NaN         NaN  
                          2018              NaN    4.521487  
                          2020              NaN   19.044016  
                          2023              NaN   23.996186  
                          2025              NaN    6.450786  
...                                         ...         ...  
Via De La Valle           2032              NaN         NaN  
                          2035              NaN         NaN  
                          2040              NaN         NaN  
                          2045              NaN         NaN  
                          2050              NaN         NaN  

[1131 rows x 8 columns]

In [125]:
# DS38 Pop Percent Change
pop_pct_change = pct_diff_df_cpa[['pop_DS38']]
over_20_pct_change_pop = pop_pct_change[(pop_pct_change['pop_DS38']>20) | (pop_pct_change['pop_DS38']<-20)]
over_20_pct_change_pop

pop_DS38
cpa_geozone               year_DS42           
32nd Street Naval Station 2023       25.728885
Barrio Logan              2023       51.044497
College Area              2023       21.573845
Downtown                  2018       21.531459
Kearny Mesa               2029       21.198157
Lindbergh Field           2029       32.338845
Midway-Pacific Highway    2020       20.440564
                          2029       22.838636
NCFUA Reserve             2050      -20.833333
Old San Diego             2018       28.922495
                          2023       55.728155
                          2025       36.066085
                          2029       54.202436
Otay                      2040       20.761364

In [126]:
# over_20_pct_change_pop.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4238_v2_results/CPA/over_20_pct_change_pop.csv', index = True)

In [129]:
# DS42 Pop Percent Change
pop_pct_change = pct_diff_df_cpa[['pop_DS42']]
over_20_pct_change_pop = pop_pct_change[(pop_pct_change['pop_DS42']>20) | (pop_pct_change['pop_DS42']<-20)]
over_20_pct_change_pop

pop_DS42
cpa_geozone               year_DS42           
32nd Street Naval Station 2023       25.159936
Barrio Logan              2023       49.829642
College Area              2023       23.480057
Downtown                  2018       21.608335
Kearny Mesa               2029       22.150295
Lindbergh Field           2023       20.618557
                          2029       33.108428
Midway-Pacific Highway    2029       23.157975
NCFUA Reserve             2029      -27.586207
                          2050       22.222222
Old San Diego             2018       25.267801
                          2023       57.437408
                          2025        35.01715
                          2029       55.290033
                          2035       20.524384
Otay                      2023       20.389083

In [130]:
# over_20_pct_change_pop.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4238_v2_results/CPA/over_20_pct_change_pop_DS42.csv', index = True)

In [131]:
# DS42 hhp Percent Change
hhp_pct_change = pct_diff_df_cpa[['hhp_DS42']]
over_20_pct_change_pop = hhp_pct_change[(hhp_pct_change['hhp_DS42']>20) | (hhp_pct_change['hhp_DS42']<-20)]
over_20_pct_change_pop

hhp_DS42
cpa_geozone               year_DS42           
32nd Street Naval Station 2023        44.60177
Barrio Logan              2023       79.086698
College Area              2023       26.323047
Downtown                  2018        24.02877
Harbor                    2018      -20.293555
Kearny Mesa               2029       22.209295
Lindbergh Field           2023       85.585586
                          2025       48.341424
                          2029       65.796283
                          2035       21.471305
Midway-Pacific Highway    2018       28.222997
                          2020       28.474379
                          2025       22.889414
                          2029       28.368927
Miramar Air Station       2025       35.878707
NCFUA Reserve             2029      -27.586207
                          2050       22.222222
Old San Diego             2018       26.329787
                          2023       60.030785
                          2025        36.00513
                          2029       56.301997
                          2035       20.715571
Otay                      2023       79.756188
                          2025       45.581587
                          2029       35.807338
                          2035       27.490621
                          2040       28.014822

In [134]:
over_20_pct_change_pop.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4238_v2_results/CPA/over_20_pct_change_hhp_DS42.csv', index = True)

In [135]:
# DS38 hhp Percent Change
hhp_pct_change = pct_diff_df_cpa[['hhp_DS38']]
over_20_pct_change_pop = hhp_pct_change[(hhp_pct_change['hhp_DS38']>20) | (hhp_pct_change['hhp_DS38']<-20)]
over_20_pct_change_pop

hhp_DS38
cpa_geozone               year_DS42           
32nd Street Naval Station 2023       55.904059
Barrio Logan              2023       81.987306
College Area              2023       24.078268
Downtown                  2018       23.938863
Kearny Mesa               2029        21.25459
Lindbergh Field           2023       74.544128
                          2025       48.015044
                          2029        65.05182
                          2035        21.26231
Midway-Pacific Highway    2018       28.459433
                          2020       29.933159
                          2025       22.799745
                          2029       27.942753
Miramar Air Station       2025       36.464933
NCFUA Reserve             2050      -20.833333
Old San Diego             2018        30.18617
                          2023        58.21501
                          2025       37.083333
                          2029       55.171713
                          2035       20.087291
Otay                      2023       76.380952
                          2025       48.855292
                          2029       35.192871
                          2035       30.491156
                          2040       29.598229

In [137]:
# over_20_pct_change_pop.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4238_v2_results/CPA/over_20_pct_change_hhp_DS38.csv', index = True)

# Check the Jurisdiction Level

In [66]:
jur_diff_df = pd.DataFrame(index=jur_df.index)

In [67]:
#41 minus 35
jur_diff_df['pop_diff'] = jur_df['pop_DS42'] - jur_df['pop_DS38']
jur_diff_df['hhp_diff'] = jur_df['hhp_DS42'] - jur_df['hhp_DS38']
jur_diff_df['gq_civ_diff'] = jur_df['gq_civ_DS42'] - jur_df['gq_civ_DS38']
jur_diff_df['gq_mil_diff'] = jur_df['gq_mil_DS42'] - jur_df['gq_mil_DS38']

In [68]:
jur_diff_df

pop_diff  hhp_diff  gq_civ_diff  gq_mil_diff
jur_geozone year_DS42                                              
Carlsbad    2016              0         0            0            0
            2018            -32       -32            0            0
            2020            -42       -42            0            0
            2023            -17       -17            0            0
            2025            -91       -91            0            0
...                         ...       ...          ...          ...
Vista       2032            323       323            0            0
            2035            328       328            0            0
            2040            399       399            0            0
            2045            140       140            0            0
            2050           -242      -242            0            0

[247 rows x 4 columns]

In [138]:
# jur_diff_df.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4238_v2_results/Jur/Jur_DS4238_diff.csv', index = True)

In [70]:
# 41 minus 35
jur_stats = pd.DataFrame(columns=jur_diff_df.columns, index=['Max', 'Min', 'Average'])
for col in jur_stats.columns:
    jur_stats[col]['Max'] = np.max(list(jur_diff_df[col]))
    jur_stats[col]['Min'] = np.min(list(jur_diff_df[col]))
    jur_stats[col]['Average'] = np.mean(list(jur_diff_df[col]))
jur_stats

,pop_diff,hhp_diff,gq_civ_diff,gq_mil_diff
Max,1481,1481,0,0
Min,-784,-784,0,0
Average,39.534413,39.534413,0.0,0.0


In [139]:
# jur_stats.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4238_v2_results/Jur/jur_stats.csv', index = True)

# Checking the sums

In [72]:
jur_names = []
for name in jur_df.index:
    jur_names.append(name[0])
unique_jur_names = list(np.unique(np.array(jur_names)))
years = ['2016','2018','2020','2023','2025','2026','2029','2030','2032','2035','2040','2045','2050']

In [73]:
jur_sums_df = pd.DataFrame(columns=jur_df.columns, index=years)
for col in jur_df:
    for year in years:
            total = 0
            for jur in unique_jur_names:
                total += int(jur_df.loc[jur].loc[[year]][col])
            jur_sums_df[col][year] = total

In [74]:
jur_sums_df

,pop_DS38,hhp_DS38,gq_civ_DS38,gq_mil_DS38,pop_DS42,hhp_DS42,gq_civ_DS42,gq_mil_DS42
2016,3906368,3732674,77690,96004,3906368,3732674,77690,96004
2018,3958161,3770063,84651,103447,3957598,3769500,84651,103447
2020,3998163,3800299,88377,109487,3998048,3800184,88377,109487
2023,4065934,3856416,90971,118547,4065298,3855780,90971,118547
2025,4114069,3900959,91543,121567,4113096,3899986,91543,121567
2026,4135148,3921752,91829,121567,4135432,3922036,91829,121567
2029,4194517,3980263,92687,121567,4195759,3981505,92687,121567
2030,4212091,3997551,92973,121567,4214017,3999477,92973,121567
2032,4244865,4030159,93139,121567,4247170,4032464,93139,121567
2035,4293438,4078483,93388,121567,4296425,4081470,93388,121567


In [140]:
jur_sums_df.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4238_v2_results/Jur/jur_sums_df.csv', index = True)

In [77]:
# Major pop Diffs
jur_pop_diff = (jur_df['pop_DS42'] - jur_df['pop_DS38']).to_frame(name='pop_diff')
jur_large_diff_pop = jur_pop_diff[(jur_pop_diff['pop_diff'] > 500) | (jur_pop_diff['pop_diff'] < -500)]

In [78]:
jur_large_diff_pop

pop_diff
jur_geozone    year_DS42          
San Diego      2023           -611
               2025           -784
               2030            846
               2032            790
               2035            741
Unincorporated 2029           1178
               2030           1162
               2032            856
               2035           1347
               2040           1348
               2045           1481

In [141]:
# jur_large_diff_pop.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4238_v2_results/Jur/jur_large_diff_pop.csv', index = True)

In [80]:
# Major hhp Diffs
jur_hhp_diff = (jur_df['hhp_DS42'] - jur_df['hhp_DS38']).to_frame(name='hhp_diff')
jur_large_diff_hhp = jur_hhp_diff[(jur_hhp_diff['hhp_diff'] > 750) | (jur_hhp_diff['hhp_diff'] < -750)]

In [81]:
jur_large_diff_hhp

hhp_diff
jur_geozone    year_DS42          
San Diego      2025           -784
               2030            846
               2032            790
Unincorporated 2029           1178
               2030           1162
               2032            856
               2035           1347
               2040           1348
               2045           1481

In [142]:
jur_large_diff_hhp.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4238_v2_results/Jur/jur_large_diff_hhp.csv', index = True)

In [83]:
jur_names = []
for name in jur_df.index:
    jur_names.append(name[0])
unique_jur_names = list(np.unique(np.array(jur_names)))
years = ['2016','2018','2020','2023','2025','2026','2029','2030','2032','2035','2040','2045','2050']


pct_diff_df_jur = pd.DataFrame(columns=jur_df.columns, index=jur_df.index)
# Year over year increase 
for zone_name in unique_jur_names:
    for i in range(1,len(years)):
        for col in jur_df.columns:
            current_val = jur_df.loc[zone_name].loc[years[i]][col]
            prior_val = jur_df.loc[zone_name].loc[years[i-1]][col]
            percent_diff = ((current_val - prior_val)/prior_val)*100
            pct_diff_df_jur.loc[zone_name].loc[years[i]][col] = percent_diff
pct_diff_df_jur

Maximum_percent_difference = pd.DataFrame(pct_diff_df_jur.max())
Minimum_percent_difference = pd.DataFrame(pct_diff_df_jur.min())


In [85]:
Maximum_percent_difference = pd.DataFrame(pct_diff_df_jur.max())
Minimum_percent_difference = pd.DataFrame(pct_diff_df_jur.min())

In [86]:
# Maximum_percent_difference.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS42_42/DS4235_results/jur/jur_DS4235_max_diff.csv', index = True)

In [87]:
# Minimum_percent_difference.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS42_42/DS4235_results/jur/jur_DS4235_min_diff.csv', index = True)

In [143]:
# DS38 hhp Percent Change
hhp_pct_change = pct_diff_df_jur[['hhp_DS38']]
over_20_pct_change_pop = hhp_pct_change[(hhp_pct_change['hhp_DS38']>20) | (hhp_pct_change['hhp_DS38']<-20)]
over_20_pct_change_pop

,,hhp_DS38
jur_geozone,year_DS42,


In [144]:
# DS42 hhp Percent Change
hhp_pct_change = pct_diff_df_jur[['hhp_DS42']]
over_20_pct_change_pop = hhp_pct_change[(hhp_pct_change['hhp_DS42']>20) | (hhp_pct_change['hhp_DS42']<-20)]
over_20_pct_change_pop

,,hhp_DS42
jur_geozone,year_DS42,


In [146]:
# DS38 pop Percent Change
pop_pct_change = pct_diff_df_jur[['pop_DS38']]
over_20_pct_change_pop = pop_pct_change[(pop_pct_change['pop_DS38']>20) | (pop_pct_change['pop_DS38']<-20)]
over_20_pct_change_pop

,,pop_DS38
jur_geozone,year_DS42,


In [147]:
# DS42 pop Percent Change
pop_pct_change = pct_diff_df_jur[['pop_DS42']]
over_20_pct_change_pop = pop_pct_change[(pop_pct_change['pop_DS42']>20) | (pop_pct_change['pop_DS42']<-20)]
over_20_pct_change_pop

,,pop_DS42
jur_geozone,year_DS42,
